# GNN_goal_recognizer

> Aggiungi citazione



##Installations

In [1]:
'''
!pip install torch_geometric
!pip install wandb -qU
!pip install networkx==3.4.2
pip install torchinfo
'''

'\n!pip install torch_geometric\n!pip install wandb -qU\n!pip install networkx==3.4.2\npip install torchinfo\n'

##Utils

In [2]:
# Helper Functions for Visualization

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.data import Data


def visualize_graph(
    title,
    G,
    color,
    labels=None,
    train_mask=None,
    pred_mask=None,
    node_feature=False,
    edge_attr=None,
    node_size=100,
    img_size=(8, 8)
):
    import matplotlib.pyplot as plt
    import networkx as nx
    import numpy as np

    # Aggiungi pesi agli archi se presenti
    if edge_attr is not None:
        for i, (u, v) in enumerate(G.edges()):
            G[u][v]['weight'] = edge_attr[i].item() if hasattr(edge_attr[i], 'item') else edge_attr[i]

    plt.figure(figsize=img_size)
    plt.title(title)

    pos = nx.spring_layout(G, seed=42)
    nx.draw_networkx(
        G, pos, with_labels=node_feature,
        node_size=node_size, node_color=color, cmap="Set2"
    )

    # Evidenzia goal reali (train_mask)
    if train_mask is not None:
        nx.draw_networkx_nodes(
            G, pos=pos,
            nodelist=[i for i, t in enumerate(train_mask) if t],
            node_color=[c for c, t in zip(color, train_mask) if t],
            cmap="Set2", edgecolors='red', linewidths=2, node_size=node_size
        )

    # Evidenzia goal predetto (pred_mask)
    if pred_mask is not None:
        nx.draw_networkx_nodes(
            G, pos=pos,
            nodelist=[i for i, t in enumerate(pred_mask) if t],
            node_color=[c for c, t in zip(color, pred_mask) if t],
            cmap="Set2", edgecolors='lime', linewidths=3, node_size=node_size
        )

    # Disegna etichette degli archi (se ci sono pesi)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    if len(edge_labels) > 0:
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

    # Legenda
    legend_handles = []
    legend_labels = []

    if train_mask is not None and any(train_mask):
        legend_handles.append(plt.Line2D([0], [0], marker='o', color='w',
                                         markeredgecolor='red', markersize=10, label='Goal reale'))
        legend_labels.append('Goal reale')
    if pred_mask is not None and any(pred_mask):
        legend_handles.append(plt.Line2D([0], [0], marker='o', color='w',
                                         markeredgecolor='lime', markersize=10, label='Goal predetto'))
        legend_labels.append('Goal predetto')

    if legend_handles:
        plt.legend(handles=legend_handles, loc='upper right')

    plt.show()


def visualize_embedding(title, h, color, labels):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])
    plt.title(title)

    scatter = plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.legend(handles=scatter.legend_elements()[0], labels=labels.values())
    plt.show()

/home/deeplearning/Pollastri_Rusmini_Lizza/path_dataset_generator/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from torch_geometric.data import Data
import torch.optim as optim

##Classes

###Map

In [4]:
#map class

from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class Map:
    V: List[List[Tuple[int, int, int]]]
    E: List[Tuple[int, int]]
    Y: List[List[int]]
    O: List[List[int]]
    Optimality: List[float]
    AvgLength: float
    ObstaclePerc: float


In [5]:
from torch_geometric.data import InMemoryDataset

class MyDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        super().__init__('.', transform)
        self.data, self.slices = self.collate(data_list)

    @property
    def num_features(self):
        return self[0].num_node_features

    @property
    def num_classes(self):
        y_all = torch.cat([d.y for d in self])
        return int(y_all.max().item() + 1)


###


###MapLoader

In [6]:
#from google.colab import drive
import json

#drive.mount('/content/drive')

def load_maps_from_drive_unified(type,size):
    """
    Carica tutte le istanze di Map insieme dal file unified
    Restituisce una lista di oggetti Map.
    """
    if size not in [8, 16, 32, 128]:
        print("size deve essere 16, 32 o 128")
        return []

    all_maps = []
    #file_path = f"/content/drive/MyDrive/Progetto_deep_learning/Dataset/{type}{size}.json"
    file_path = f"{type}{size}.json"
    try:
        with open(file_path, "r") as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        print(f"File {i} non trovato o non leggibile")
        return None

    if not data:
        print(f"File {i} vuoto")
        return None



    all_maps.extend([Map(**map_dict) for map_dict in data])

    return all_maps


##Creation and loading of the dataset

In [7]:
from torch_geometric.data import Dataset, Data
import torch

class GoalRecognitionDataset(Dataset):
    def __init__(self, maps, transform=None):
        """
        maps: lista di mappe (con la griglia + edge_index già costruito)
        paths: lista di percorsi (ogni percorso è lista di nodi [src, ..., goal])
        """
        super().__init__(None, transform)
        self.maps = maps

    '''
    @percentage è la percentuale di percorso che passo al metodo, cioè quanto è lungo il percorso delle varie entries
    prima generavamo tutti i percorsi aumentati di uno alla volta, ora posso scegliere quanto farlo lungo
    '''
    def generate_entries(self,map_index, path_index,print_or_not, percentage=100):
        """
        Genera le entry per ogni sottopercorso (minimo 2 elementi) di un path di O.
        Ogni entry è (V_mod, E, Y).
        """
        entries = []

        len_entr = len(self.maps[map_index].O[path_index])
        new_len = int((100-percentage)*len_entr/100)
        if new_len < 2:
          new_len = 2
        #for step in range(2, len(self.maps[map_index].O[path_index])+1):
        for step in range(new_len, len_entr+1):
            V_mod = [row.copy() for row in self.maps[map_index].V[path_index]]

            visited = self.maps[map_index].O[path_index][:step-1]         # nodi già visitati
            agent_pos = self.maps[map_index].O[path_index][step-1]        # posizione attuale
            future = self.maps[map_index].O[path_index][step:]            # nodi futuri -> liberi

            if(print_or_not):
              print(visited)

            for v in visited:
                V_mod[v] = [0,0,1]

            V_mod[agent_pos] = [0,1,0]

            for f in future:
                V_mod[f] = [1,0,0]

            x = torch.tensor(V_mod, dtype=torch.float)
            edge_index = torch.tensor(self.maps[map_index].E, dtype=torch.long).t().contiguous()


            y = torch.tensor(self.maps[map_index].Y[path_index], dtype=torch.float)

            # Esponenziazione per aumentare il contrasto
            alpha = 1.0
            y_transformed = y ** alpha

            # Normalizza di nuovo
            y_transformed = y_transformed / y_transformed.sum()

            # Ora puoi usarlo come target
            entries.append(Data(x=x, edge_index=edge_index, y=y_transformed))

        return entries

    def generate_all_entries(self):
        """
        Genera tutte le entry per tutti i percorsi di tutte le mappe.
        """
        entries = []

        for map in range(len(self.maps)):
            for path in range(len(self.maps[map].O)):
                for entry in self.generate_entries(map, path,False, 100): #30 da modificare se voglio sottopercorsi più o meno lunghi
                    entries.append(entry)

        return entries

CARICA TUTTE LE MAPPE

In [8]:

import torch
from torch_geometric.data import Data

size=8
maps=load_maps_from_drive_unified("DfsRandomJump",size)


In [9]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

goalRecognitionDataset = GoalRecognitionDataset(maps)
dataset = goalRecognitionDataset.generate_all_entries()

# Suddivisione in train / val / test (80% / 10% / 10%)
total_len = len(dataset)
train_len = int(0.8 * total_len)
val_len   = int(0.1 * total_len)
test_len  = total_len - train_len - val_len

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_len, val_len, test_len])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


##Logging WanDB


In [10]:
# Log in to your W&B account
import wandb
import random
import math

In [11]:
wandb.login(key="38de765b09e71e9b6b33218b7ade62f2349d81c0")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/deeplearning/.netrc
wandb: Currently logged in as: m-lizza002 (m-lizza002-university-of-brescia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

##Sweeps

###Define a sweep

In [12]:
parameters_dict = {
    'optimizer': {
        'values': ['Adam', 'SGD']
      },
    'dropout': {
          'values': [0, 0.3, 0.4, 0.5]
      },
    'epochs': {
        'value': 100
      },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms
        #'distribution': 'q_log_uniform_values',
        #'q': 8,
        #'min': 1,
        #'max': 1,
        'value': 16
      },
    'in_channels': {
        'value': 3
      },
    'hidden_channels_1': {
        'values': [128]
      },
    'hidden_channels_2': {
        'values': [64]
      }
}


In [13]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': parameters_dict
    }

In [14]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_acc'},
 'parameters': {'batch_size': {'value': 16},
                'dropout': {'values': [0, 0.3, 0.4, 0.5]},
                'epochs': {'value': 100},
                'hidden_channels_1': {'values': [128]},
                'hidden_channels_2': {'values': [64]},
                'in_channels': {'value': 3},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['Adam', 'SGD']}}}


###Initialize the Sweep

In [ ]:
#sweep_id = wandb.sweep(sweep_config, project="Node_Classificator")

Create sweep with ID: rzb8yah3
Sweep URL: https://wandb.ai/m-lizza002-university-of-brescia/Node_Classificator/sweeps/rzb8yah3


##Training


### Node Classificator

In [16]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

SAVE_DIR = "./gnn/"
MODEL_NAME = "node_cls_soft.pth"

# Ho aggiornato i valori '0' e 'None' a None o al tipo corretto per coerenza,
# ma i parametri effettivi verranno aggiornati nel training.
best_params = {
    'optimizer': {'value': None}, 
    'dropout': {'value': 0.0},
    'epochs': {'value': 0},
    'learning_rate': {'value': 0.0},
    'batch_size': {'value': None},
    'in_channels': {'value': 0},
    'hidden_channels_1': {'value': 0},
    'hidden_channels_2': {'value': 0}
}


class NodeClassificator(nn.Module):
    """
    GNN per predire distribuzioni di probabilità/score nodo-wise (soft targets, es. vicinanza al goal).
    Input: x [num_nodes, in_channels], edge_index [2, num_edges]
    Output: logits [num_nodes] (score non vincolato, Sigmoid applicata all'esterno per le probabilità 0..1)
    """
    def __init__(self, in_channels, hidden_channels_1, hidden_channels_2, dropout=0.0):
        super(NodeClassificator, self).__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels_1)
        self.conv2 = GCNConv(hidden_channels_1, hidden_channels_2)
        # Output 1 canale per predire un singolo score (vicinanza al goal) per nodo
        self.lin = nn.Linear(hidden_channels_2, 1) 
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, edge_index):
        h = self.relu(self.conv1(x, edge_index))
        h = self.dropout(h)
        h = self.relu(self.conv2(h, edge_index))
        logits = self.lin(h).squeeze(-1)  # [num_nodes]
        return logits  # score non vincolato (logit)

    # -------------------------------------------------------------------------
    # ------------------------------- TRAINING --------------------------------
    # -------------------------------------------------------------------------

    def train_gcn(self, train_loader, val_loader, optimizer, criterion, device,
                  num_epochs, patience=10):
        """
        Addestramento per soft target prediction nodo-wise.
        - Criterion: Si raccomanda nn.BCEWithLogitsLoss (o nn.MSELoss).
        - Metrica: Root Mean Square Error (RMSE) per misurare la performance di regressione.
        """
        best_val_loss = float('inf')
        epochs_no_improve = 0

        for epoch in range(num_epochs):
            self.train()
            total_train_loss = 0
            total_train_rmse = 0
            total_nodes = 0

            for batch in train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()

                logits = self(batch.x, batch.edge_index)
                
                # IMPORTANTISSIMO: Usiamo i LOGITS e il TARGET Y (che deve essere Float) 
                # e la Loss BCEWithLogitsLoss gestisce il Sigmoid internamente per stabilità.
                # Nota: batch.y deve essere di tipo torch.float e avere la stessa shape di logits [num_nodes]
                loss = criterion(logits, batch.y) 
                loss.backward()
                optimizer.step()

                total_train_loss += loss.item()
                
                # Calcolo RMSE (Root Mean Square Error) come metrica
                probs = torch.sigmoid(logits)
                # RMSE = sqrt(MSE)
                total_train_rmse += torch.sqrt(F.mse_loss(probs, batch.y, reduction='sum')).item()
                total_nodes += batch.y.numel()

            avg_train_loss = total_train_loss / len(train_loader)
            train_rmse = total_train_rmse / total_nodes 

            # Valutazione
            val_loss, val_rmse = self.eval_gcn(val_loader, criterion, device, return_metric=True)

            print(f"Epoch {epoch:03d} | Train Loss: {avg_train_loss:.4f} | Train RMSE: {train_rmse:.4f} | "
                  f"Val Loss: {val_loss:.4f} | Val RMSE: {val_rmse:.4f}")
            
            # Early stopping + salvataggio modello migliore (basato su Val Loss)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0

                global best_params # Accesso alla variabile globale
                
                # Aggiornamento parametri salvati
                best_params.update({
                    'optimizer': optimizer.__class__.__name__,
                    'dropout': self.dropout.p,
                    'epochs': epoch + 1, # Salva l'epoca attuale (che è la migliore)
                    'learning_rate': optimizer.param_groups[0]['lr'],
                    'batch_size': getattr(train_loader.dataset, 'batch_size', None) if hasattr(train_loader.dataset, 'batch_size') else train_loader.batch_size,
                    'in_channels': self.conv1.in_channels,
                    'hidden_channels_1': self.conv1.out_channels,
                    'hidden_channels_2': self.conv2.out_channels
                })

                if not os.path.exists(SAVE_DIR):
                    os.mkdir(SAVE_DIR)
                torch.save(self.state_dict(), os.path.join(SAVE_DIR, MODEL_NAME))
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

        return avg_train_loss, best_val_loss, train_rmse, val_rmse

    # -------------------------------------------------------------------------
    # ------------------------------- VALIDATION ------------------------------
    # -------------------------------------------------------------------------

    @torch.no_grad()
    def eval_gcn(self, val_loader, criterion, device, return_metric=False):
        self.eval()
        total_val_loss = 0
        total_val_rmse = 0
        total_nodes = 0

        for batch in val_loader:
            batch = batch.to(device)
            logits = self(batch.x, batch.edge_index)
            
            # Loss basata sui logits
            loss = criterion(logits, batch.y)
            total_val_loss += loss.item()

            if return_metric:
                # Calcolo RMSE (metrica di regressione)
                probs = torch.sigmoid(logits)
                total_val_rmse += torch.sqrt(F.mse_loss(probs, batch.y, reduction='sum')).item()
                total_nodes += batch.y.numel()

        avg_val_loss = total_val_loss / len(val_loader)
        val_rmse = total_val_rmse / total_nodes if return_metric else None
        
        if return_metric:
            return avg_val_loss, val_rmse
        return avg_val_loss

    # -------------------------------------------------------------------------
    # -------------------------------- PREDICT --------------------------------
    # -------------------------------------------------------------------------

    @torch.no_grad()
    def predict(self, test_loader, device):
        """
        Ritorna le probabilità (0..1) di vicinanza al goal per ogni nodo.
        """
        self.eval()
        all_probs = []

        for batch in test_loader:
            batch = batch.to(device)
            logits = self(batch.x, batch.edge_index)
            # Trasformiamo i logits in probabilità (0..1) usando la Sigmoid
            probs = torch.sigmoid(logits) 
            all_probs.append(probs.cpu().tolist())

        # Ritorna una lista di liste/tensori di probabilità, una per ogni batch
        return all_probs

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim # Necessario per getattr(optim, config.optimizer)
import wandb

# Assumiamo che train_loader e val_loader siano definiti globalmente o passati come argomenti
# def train_sweep(config=None, train_loader=None, val_loader=None): # Versione più pulita

def train_sweep(config=None):
    with wandb.init(config=config) as run:
        config = wandb.config

        run_name = (
            f"{config.optimizer}"
            f"-lr{config.learning_rate:.4f}"
            f"-bs{config.batch_size}"
            f"-drop{config.dropout}"
            f"-h1{config.hidden_channels_1}"
            f"-h2{config.hidden_channels_2}"
        )
        run.name = run_name

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # 1. Inizializzazione del Modello
        model = NodeClassificator(
            in_channels=config.in_channels,
            hidden_channels_1=config.hidden_channels_1,
            hidden_channels_2=config.hidden_channels_2,
            dropout=config.dropout
        ).to(device)

        # 2. Inizializzazione dell'Optimizer
        optimizer = getattr(optim, config.optimizer)(
            model.parameters(), lr=config.learning_rate
        )

        # 3. MODIFICA CRITERION (Da KLDivLoss a BCEWithLogitsLoss)
        # BCEWithLogitsLoss è ideale per target 0..1 e logits, 
        # garantendo stabilità numerica e gestendo la Sigmoid internamente.
        criterion = torch.nn.BCEWithLogitsLoss() 

        # 4. Avvio del Training
        model.train_gcn(
            train_loader=train_loader, # Assumiamo sia disponibile
            val_loader=val_loader,     # Assumiamo sia disponibile
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            num_epochs=config.epochs,
        )

        # Stampa dei parametri migliori (aggiornati all'interno di model.train_gcn)
        print("BEST PARAMS:", best_params)

In [18]:
"""
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

SAVE_DIR = "./gnn/"
MODEL_NAME = "node_cls_soft.pth"

best_params = {
    'optimizer': {'values': ['Adam']},
    'dropout': {'value': 0.0},
    'epochs': {'value': 0},
    'learning_rate': {'value': 0.0},
    'batch_size': {'value': 0},
    'in_channels': {'value': 0},
    'hidden_channels_1': {'value': 0},
    'hidden_channels_2': {'value': 0}
}


class NodeClassificator(nn.Module):
    
    GNN per predire distribuzioni di probabilità nodo-wise (soft targets).
    Input: x [num_nodes, in_channels], edge_index [2, num_edges]
    Output: logits [num_nodes] (log_softmax applicato nel training)
    
    def __init__(self, in_channels, hidden_channels_1, hidden_channels_2, dropout=0.0):
        super(NodeClassificator, self).__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels_1)
        self.conv2 = GCNConv(hidden_channels_1, hidden_channels_2)
        self.lin = nn.Linear(hidden_channels_2, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, edge_index):
        h = self.relu(self.conv1(x, edge_index))
        h = self.dropout(h)
        h = self.relu(self.conv2(h, edge_index))
        logits = self.lin(h).squeeze(-1)  # [num_nodes]
        return logits  # log_softmax/applicato all'esterno

    # -------------------------------------------------------------------------
    # ------------------------------- TRAINING --------------------------------
    # -------------------------------------------------------------------------

    def train_gcn(self, train_loader, val_loader, optimizer, criterion, device,
                  num_epochs, patience=10):

        best_val_loss = float('inf')
        epochs_no_improve = 0

        for epoch in range(num_epochs):
            self.train()
            total_train_loss = 0
            total_correct = 0
            total_nodes = 0

            for batch in train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()

                logits = self(batch.x, batch.edge_index)
                log_probs = F.log_softmax(logits, dim=0)
                loss = criterion(log_probs, batch.y)
                loss.backward()
                optimizer.step()

                total_train_loss += loss.item()

                # Calcolo accuracy: confronta argmax di pred e target
                pred_nodes = torch.argmax(F.softmax(logits, dim=0), dim=0)
                target_nodes = torch.argmax(batch.y, dim=0)
                if pred_nodes == target_nodes:
                    total_correct += 1
                total_nodes += 1

            avg_train_loss = total_train_loss / len(train_loader)
            train_acc = total_correct / total_nodes

            val_loss, val_acc = self.eval_gcn(val_loader, criterion, device, return_acc=True)

            print(f"Epoch {epoch:03d} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

            # Early stopping + salvataggio modello migliore
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0

                best_params.update({
                    'optimizer': optimizer.__class__.__name__,
                    'dropout': self.dropout.p,
                    'epochs': num_epochs,
                    'learning_rate': optimizer.param_groups[0]['lr'],
                    'batch_size': getattr(train_loader.dataset, 'batch_size', None),
                    'in_channels': self.conv1.in_channels,
                    'hidden_channels_1': self.conv1.out_channels,
                    'hidden_channels_2': self.conv2.out_channels
                })

                if not os.path.exists(SAVE_DIR):
                    os.mkdir(SAVE_DIR)
                torch.save(self.state_dict(), os.path.join(SAVE_DIR, MODEL_NAME))
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

        return avg_train_loss, best_val_loss, train_acc, val_acc

    # -------------------------------------------------------------------------
    # ------------------------------- VALIDATION ------------------------------
    # -------------------------------------------------------------------------

    @torch.no_grad()
    def eval_gcn(self, val_loader, criterion, device, return_acc=False):
        self.eval()
        total_val_loss = 0
        total_correct = 0
        total_nodes = 0

        for batch in val_loader:
            batch = batch.to(device)
            logits = self(batch.x, batch.edge_index)
            log_probs = F.log_softmax(logits, dim=0)
            loss = criterion(log_probs, batch.y)
            total_val_loss += loss.item()

            if return_acc:
                pred_nodes = torch.argmax(F.softmax(logits, dim=0), dim=0)
                target_nodes = torch.argmax(batch.y, dim=0)
                if pred_nodes == target_nodes:
                    total_correct += 1
                total_nodes += 1

        avg_val_loss = total_val_loss / len(val_loader)
        val_acc = total_correct / total_nodes if return_acc else None
        if return_acc:
            return avg_val_loss, val_acc
        return avg_val_loss

    # -------------------------------------------------------------------------
    # -------------------------------- PREDICT --------------------------------
    # -------------------------------------------------------------------------

    @torch.no_grad()
    def predict(self, test_loader, device):
        self.eval()
        all_probs = []

        for batch in test_loader:
            batch = batch.to(device)
            logits = self(batch.x, batch.edge_index)
            probs = F.softmax(logits, dim=0)
            all_probs.append(probs.cpu().tolist())

        return all_probs
    """

'\nimport os\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nfrom torch_geometric.nn import GCNConv\n\nSAVE_DIR = "./gnn/"\nMODEL_NAME = "node_cls_soft.pth"\n\nbest_params = {\n    \'optimizer\': {\'values\': [\'Adam\']},\n    \'dropout\': {\'value\': 0.0},\n    \'epochs\': {\'value\': 0},\n    \'learning_rate\': {\'value\': 0.0},\n    \'batch_size\': {\'value\': 0},\n    \'in_channels\': {\'value\': 0},\n    \'hidden_channels_1\': {\'value\': 0},\n    \'hidden_channels_2\': {\'value\': 0}\n}\n\n\nclass NodeClassificator(nn.Module):\n\n    GNN per predire distribuzioni di probabilità nodo-wise (soft targets).\n    Input: x [num_nodes, in_channels], edge_index [2, num_edges]\n    Output: logits [num_nodes] (log_softmax applicato nel training)\n\n    def __init__(self, in_channels, hidden_channels_1, hidden_channels_2, dropout=0.0):\n        super(NodeClassificator, self).__init__()\n\n        self.conv1 = GCNConv(in_channels, hidden_channels_1)\n        self.conv2

In [19]:
"""
def train_sweep(config=None):
    with wandb.init(config=config) as run:
        config = wandb.config

        run_name = (
            f"{config.optimizer}"
            f"-lr{config.learning_rate:.4f}"
            f"-bs{config.batch_size}"
            f"-drop{config.dropout}"
            f"-h1{config.hidden_channels_1}"
            f"-h2{config.hidden_channels_2}"
        )
        run.name = run_name

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model = NodeClassificator(
            in_channels=config.in_channels,
            hidden_channels_1=config.hidden_channels_1,
            hidden_channels_2=config.hidden_channels_2,
            dropout=config.dropout
        ).to(device)

        optimizer = getattr(optim, config.optimizer)(
            model.parameters(), lr=config.learning_rate
        )

        criterion = torch.nn.KLDivLoss(reduction='batchmean')

        model.train_gcn(
            train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            num_epochs=config.epochs,
        )

        print("BEST PARAMS:", best_params)
"""

'\ndef train_sweep(config=None):\n    with wandb.init(config=config) as run:\n        config = wandb.config\n\n        run_name = (\n            f"{config.optimizer}"\n            f"-lr{config.learning_rate:.4f}"\n            f"-bs{config.batch_size}"\n            f"-drop{config.dropout}"\n            f"-h1{config.hidden_channels_1}"\n            f"-h2{config.hidden_channels_2}"\n        )\n        run.name = run_name\n\n        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\n        model = NodeClassificator(\n            in_channels=config.in_channels,\n            hidden_channels_1=config.hidden_channels_1,\n            hidden_channels_2=config.hidden_channels_2,\n            dropout=config.dropout\n        ).to(device)\n\n        optimizer = getattr(optim, config.optimizer)(\n            model.parameters(), lr=config.learning_rate\n        )\n\n        criterion = torch.nn.KLDivLoss(reduction=\'batchmean\')\n\n        model.train_gcn(\n            train_loa

In [26]:
sweep_id = "rzb8yah3"

if wandb.run is not None:
    wandb.finish()

wandb.agent(sweep_id, function=train_sweep, count=5)

wandb: Agent Starting Run: 0ft17v56 with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	hidden_channels_1: 128
wandb: 	hidden_channels_2: 64
wandb: 	in_channels: 3
wandb: 	learning_rate: 0.06931744537926125
wandb: 	optimizer: SGD


Epoch 000 | Train Loss: 0.0992 | Train RMSE: 0.0005 | Val Loss: 0.0952 | Val RMSE: 0.0005
Epoch 001 | Train Loss: 0.0954 | Train RMSE: 0.0005 | Val Loss: 0.0952 | Val RMSE: 0.0004
Epoch 002 | Train Loss: 0.0952 | Train RMSE: 0.0005 | Val Loss: 0.0951 | Val RMSE: 0.0004
Epoch 003 | Train Loss: 0.0952 | Train RMSE: 0.0004 | Val Loss: 0.0951 | Val RMSE: 0.0004
Epoch 004 | Train Loss: 0.0951 | Train RMSE: 0.0004 | Val Loss: 0.0950 | Val RMSE: 0.0004
Epoch 005 | Train Loss: 0.0950 | Train RMSE: 0.0004 | Val Loss: 0.0950 | Val RMSE: 0.0004
Epoch 006 | Train Loss: 0.0950 | Train RMSE: 0.0004 | Val Loss: 0.0949 | Val RMSE: 0.0004
Epoch 007 | Train Loss: 0.0950 | Train RMSE: 0.0004 | Val Loss: 0.0949 | Val RMSE: 0.0004
Epoch 008 | Train Loss: 0.0949 | Train RMSE: 0.0004 | Val Loss: 0.0949 | Val RMSE: 0.0004
Epoch 009 | Train Loss: 0.0949 | Train RMSE: 0.0004 | Val Loss: 0.0949 | Val RMSE: 0.0004
Epoch 010 | Train Loss: 0.0949 | Train RMSE: 0.0004 | Val Loss: 0.0949 | Val RMSE: 0.0004
Epoch 011 

: 

In [ ]:
print(best_params)

In [ ]:
#print(model)


In [ ]:

import torch

# scegli device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mapIndex=32
pathIndex=9
goalRecognitionDataset = GoalRecognitionDataset(maps)
map_and_paths= goalRecognitionDataset.generate_entries(mapIndex,pathIndex,True)

In [ ]:
from torch_geometric.utils import to_networkx

best_model = NodeClassificator(
    in_channels=best_params['in_channels'],
    hidden_channels_1=best_params['hidden_channels_1'],
    hidden_channels_2=best_params['hidden_channels_2'],
    dropout=best_params['dropout']
).to(device)

# Carica i pesi
MODEL_PATH = os.path.join(SAVE_DIR, MODEL_NAME)
best_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
#best_model.eval()


resulted_goals = []
for paths in map_and_paths:

    data = paths.to(device)
    best_model.eval()
    with torch.no_grad():
        out = best_model(data.x, data.edge_index)

    probs = torch.sigmoid(out).squeeze()

    G = to_networkx(data, to_undirected=True)

    node_states = torch.argmax(data.x, dim=1).cpu().numpy()
    is_goal = data.y.cpu().numpy() > 0.5

    pred_idx = probs.argmax()
    pred_onehot = torch.zeros_like(probs)
    pred_onehot[pred_idx] = 1
    resulted_goals.append(pred_idx.item())

    print(f"Probabilità del goal predetto: {probs[pred_idx]:.4f}")

    predicted_mask = np.zeros(len(is_goal), dtype=bool)
    predicted_mask[pred_idx] = True

    visualize_graph(
        "Graph - Robot States and Goals",
        G,
        color=node_states,
        train_mask=is_goal,       # bordo rosso per i goal reali
        pred_mask=predicted_mask, # bordo verde (ad esempio) per il goal predetto
        node_size=300
    )

print(f"Numero predizioni {len(resulted_goals)}")
print(f"Goals predetti:\n{resulted_goals}\n")

print(f"Lunghezza percorso {len(maps[mapIndex].O[pathIndex])}")
print(f"Percorso compiuto:\n{maps[mapIndex].O[pathIndex]}\n\n")


In [ ]:
"""
from torch_geometric.utils import to_networkx
import numpy as np
import torch

best_model = NodeClassificator(
    in_channels=best_params['in_channels'],
    hidden_channels_1=best_params['hidden_channels_1'],
    hidden_channels_2=best_params['hidden_channels_2'],
    linear_channels=best_params['linear_channels'],
    dropout=best_params['dropout']
).to(device)

MODEL_PATH = os.path.join(SAVE_DIR, MODEL_NAME)
best_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
best_model.eval()

resulted_goals = []
resulted_paths = []

print("\n--- INIZIO PREDIZIONE PROGRESSIVA ---\n")

for i, data in enumerate(map_and_paths):
    data = data.to(device)

    # Nodo iniziale del robot (posizione corrente)
    current_node = torch.argmax(data.x[:, 1]).item()
    visited_nodes = set([current_node])
    predicted_path = [current_node]

    print(data)

    print(f"\n🗺️  Mappa {i+1}/{len(map_and_paths)} - Nodo iniziale: {current_node}")

    while True:
        # --- Aggiorna feature dei nodi ---
        x_updated = data.x.clone()
        x_updated[:, 1] = 0               # reset posizione robot
        x_updated[current_node, 1] = 1    # nodo corrente = posizione robot
        for v in visited_nodes:
            x_updated[v, 0] = 1           # nodi visitati

        # --- Predizione del goal ---
        with torch.no_grad():
            out = best_model(x_updated, data.edge_index)
            probs = torch.sigmoid(out).squeeze().cpu()

        # Maschera nodi già visitati
        probs_masked = probs.clone()
        for v in visited_nodes:
            probs_masked[v] = -float('inf')

        # Se tutti i nodi sono stati visitati → fine
        if torch.all(probs_masked == -float('inf')):
            print("⚠️  Tutti i nodi visitati, fermo qui.")
            break

        # Nodo predetto come prossimo o goal
        pred_idx = probs_masked.argmax().item()
        pred_prob = probs[pred_idx].item()

        # Aggiorna stato
        visited_nodes.add(pred_idx)
        predicted_path.append(pred_idx)
        current_node = pred_idx

        # Se la probabilità è abbastanza alta → goal trovato
        if pred_prob > 0.5:
            print(f"🏁 Goal trovato nel nodo {pred_idx} con prob {pred_prob:.4f}")
            resulted_goals.append(pred_idx)
            break

    resulted_paths.append(predicted_path)

    # Visualizza grafo del risultato finale
    G = to_networkx(data, to_undirected=True)
    node_states = torch.argmax(x_updated, dim=1).cpu().numpy()
    predicted_mask = np.zeros(len(node_states), dtype=bool)
    predicted_mask[predicted_path[-1]] = True
    is_goal = data.y.cpu().numpy() > 0.5

    visualize_graph(
        f"Graph - Predizione finale mappa {i+1}",
        G,
        color=node_states,
        train_mask=is_goal,
        pred_mask=predicted_mask,
        node_size=300
    )

print("\n--- RISULTATI FINALI ---")
for i, (goal, path) in enumerate(zip(resulted_goals, resulted_paths)):
    print(f"Mappa {i+1}: Goal predetto = {goal}, Lunghezza percorso = {len(path)}")
    """
